In [109]:
import pandas as pd
from keras.models import Sequential
from keras.layers import Dense,  Flatten, Activation, Dropout
from keras.wrappers.scikit_learn import KerasClassifier
from keras.utils import np_utils
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.preprocessing import LabelEncoder
from sklearn.pipeline import Pipeline
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from keras.regularizers import l2

In [3]:
df = pd.read_csv("../data/full_geneexp_phase2_1004.csv")
df = df.drop(["Unnamed: 0","Unnamed: 0.1","Unnamed: 0.1.1"],axis=1)

In [110]:
def baseline_model(output_len, input_len = 978):
    model = Sequential()
    model.add(Dense(20, input_dim=input_len, activation='relu', kernel_regularizer=l2(0.01), bias_regularizer=l2(0.01)))
    model.add(Dense(10, input_dim=input_len, activation='relu', kernel_regularizer=l2(0.01), bias_regularizer=l2(0.01)))
    model.add(Dropout(0.5))
    model.add(Dense(10, input_dim=input_len, activation='relu', kernel_regularizer=l2(0.01), bias_regularizer=l2(0.01)))
    model.add(Dropout(0.5))
    model.add(Dense(5, input_dim=input_len, activation='relu', kernel_regularizer=l2(0.01), bias_regularizer=l2(0.01)))
    model.add(Dense(output_len, activation='softmax'))
    model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

In [111]:
def run(X,y,output_len):
    # output_len is number of perturbagens for that particular cell line.
    le = preprocessing.LabelEncoder()
    y1 = le.fit_transform(y)
    
    X_train, X_test, y_train, y_test = train_test_split(X, y1)
    print("X_train: ",len(X_train)," X_test: ",len(X_test))
    
    model = baseline_model(output_len)
    model.fit(X_train, y_train, epochs=100, verbose=0)
    acc = model.evaluate(X_test, y_test)
    return acc

In [112]:
# Make individual datasets for the top 10 cell lines
top10 = ['MCF7',"PC3",'HA1E','A375','HT29','YAPC','HELA']
toDrop = ['smiles','name','id','inchi_key','signature','celline']

def getATCclass(x):
    return x[0]

print("celline | targets | atc | total | atc class list")
for i in top10:
    cl = df[df['celline']==i]
    cl = cl.drop(toDrop, axis=1)
    atc_letter = cl['atc'].apply(getATCclass)
    output_len_target = len(np.unique(cl['target']))
    output_len_atc = len(np.unique(atc_letter))
    print(i,output_len_target,output_len_atc, len(cl), end=" - ")
    print(np.unique(atc_letter.tolist()))
    
    cl_dropped = cl.drop(['atc', 'target'], axis = 1)
    X = cl_dropped
    y_atc = atc_letter.tolist()
    y_target = cl['target'].tolist()
    
    acc_atc = run(X, y_atc, output_len_atc)
    acc_target = run(X, y_target, output_len_target)
    
    print("Target acc: ", acc_target)
    print("ATC acc: ", acc_atc,"\n\n")

celline | targets | atc | total | atc class list
MCF7 778 14 6630 - ['A' 'B' 'C' 'D' 'G' 'H' 'J' 'L' 'M' 'N' 'P' 'R' 'S' 'V']
X_train:  4972  X_test:  1658
1658/1658 [==============================] - 2s 1ms/step
X_train:  4972  X_test:  1658
1658/1658 [==============================] - 2s 1ms/step
Target acc:  [5.958439372560803, 0.10494571777714463]
ATC acc:  [2.550625259252164, 0.2822677927008584] 


PC3 778 14 6515 - ['A' 'B' 'C' 'D' 'G' 'H' 'J' 'L' 'M' 'N' 'P' 'R' 'S' 'V']
X_train:  4886  X_test:  1629
1629/1629 [==============================] - 2s 1ms/step
X_train:  4886  X_test:  1629
1629/1629 [==============================] - 4s 2ms/step
Target acc:  [6.0748848458460465, 0.07734806629376882]
ATC acc:  [2.820539320810504, 0.24984653139494914] 


HA1E 774 14 6414 - ['A' 'B' 'C' 'D' 'G' 'H' 'J' 'L' 'M' 'N' 'P' 'R' 'S' 'V']
X_train:  4810  X_test:  1604
1604/1604 [==============================] - 3s 2ms/step
X_train:  4810  X_test:  1604
1604/1604 [=============================

In [68]:
X = df.drop(toDrop + ['atc','target'], axis=1)
y = df['target'].tolist()
le = preprocessing.LabelEncoder()
y1 = le.fit_transform(y)

In [75]:
run(X, y1, len(np.unique(y1)))

X_train:  37029  X_test:  12344
Epoch 1/10
37029/37029 [==============================] - 11s 303us/step - loss: 4.9807 - acc: 0.1549
Epoch 2/10
37029/37029 [==============================] - 8s 218us/step - loss: 3.2367 - acc: 0.2997
Epoch 3/10
37029/37029 [==============================] - 8s 216us/step - loss: 2.4407 - acc: 0.4131
Epoch 4/10
37029/37029 [==============================] - 8s 213us/step - loss: 1.9149 - acc: 0.5095
Epoch 5/10
37029/37029 [==============================] - 8s 213us/step - loss: 1.5252 - acc: 0.5890
Epoch 6/10
37029/37029 [==============================] - 8s 213us/step - loss: 1.2274 - acc: 0.6598
Epoch 7/10
37029/37029 [==============================] - 8s 211us/step - loss: 0.9965 - acc: 0.7143
Epoch 8/10
37029/37029 [==============================] - 9s 237us/step - loss: 0.8189 - acc: 0.7624
Epoch 9/10
37029/37029 [==============================] - 10s 260us/step - loss: 0.6881 - acc: 0.79710s - loss: 
Epoch 10/10
12344/12344 [=====================

[4.537354791002156, 0.4076474400325325]

In [72]:
len(y1)

49373